In [1]:
!pip3 install pin meshcat

Defaulting to user installation because normal site-packages is not writeable


In [52]:
#
# In this short script, we show how to use RobotWrapper
# integrating different kinds of viewers
#
 
import pinocchio as pin
from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.visualize import GepettoVisualizer, MeshcatVisualizer
from sys import argv
import os
from os.path import dirname, join, abspath
from time import sleep
import numpy as np
from numpy.linalg import norm, solve


In [54]:

VISUALIZER = MeshcatVisualizer

IK_EPS = 1e-3
IK_IT_MAX = 100
IK_DT = 1e-1
IK_DAMP = 1e-12
 
# Load the URDF model with RobotWrapper
# Conversion with str seems to be necessary when executing this file with ipython
pinocchio_model_dir = join(abspath(''), "models")
 
model_path = join(pinocchio_model_dir, "example-robot-data/robots")
mesh_dir = pinocchio_model_dir
urdf_filename = "z1.urdf"
urdf_model_path = join(join(model_path, "z1_description/urdf"), urdf_filename)
 
robot = RobotWrapper.BuildFromURDF(urdf_model_path, mesh_dir)
 
# alias
model = robot.model
data = robot.data

# Get end-effector id
ee_frame_id = model.getFrameId("link06")
world_frame_id = model.getFrameId("world")
ee_joint_id = model.getFrameId("joint6")
print(f"End-effectors id: {ee_frame_id}")
 
# do whatever, e.g. compute the center of mass position expressed in the world frame
q = robot.q0
 
# Show model with a visualizer of your choice
if VISUALIZER:
    robot.setVisualizer(VISUALIZER())
    robot.initViewer()
    robot.loadViewerModel("pinocchio")
    robot.display(q)

End-effectors id: 16
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7016/static/


In [43]:
def solve_ik(oMdes: pin.SE3, q0: np.ndarray):
    q = q0.copy()
    i = 0
    while True:
        pin.forwardKinematics(model, data, q)
        iMd = data.oMi[-1].actInv(oMdes)
        err = pin.log(iMd).vector  # in joint frame
        if norm(err) < IK_EPS:
            success = True
            break
        if i >= IK_IT_MAX:
            success = False
            break
        J = pin.computeFrameJacobian(model, data, q, ee_frame_id, pin.ReferenceFrame.WORLD)  # in joint frame
        J = -np.dot(pin.Jlog6(iMd.inverse()), J)
        v = -J.T.dot(solve(J.dot(J.T) + IK_DAMP * np.eye(6), err))
        q = pin.integrate(model, q, v * IK_DT)
        i += 1
    return success, q

In [70]:
q = pin.randomConfiguration(model)
print(type(q))
J = pin.computeFrameJacobian(model, data, q, ee_frame_id, pin.ReferenceFrame.WORLD)
robot.display(q)
oMdes = pin.SE3(np.eye(3), np.array([0.3, 0.0, 0.3]))
ok, q_des = solve_ik(oMdes, q)
print(ok, q_des)

<class 'numpy.ndarray'>
True [-3.33439174e-03  1.61664871e+00 -6.11480750e-01  5.27790729e+00
  3.52530555e-03 -6.28330292e+00]


In [51]:
# Specify robot force-torque estimation precizion

def ft_precision(model, data, q: np.ndarray, torque_error: np.ndarray) -> tuple[np.float, np.float]:
    ft_error = 

In [71]:
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(data_2_ok[:,0],data_2_ok[:,1],data_2_ok[:,2], color = "green", alpha = 0.25)
# ax.scatter(data_1_ok[:,0],data_1_ok[:,1],data_1_ok[:,2], color = "yellow", alpha = 0.25)
ax.scatter([0.0], [0.0], [0.0], color = "blue")
plt.title("Configuration = [0,1,-1]")
ax.xlabel('x')
ax.ylabel('y')
ax.zlabel('z')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(data_3_ok[:,0],data_3_ok[:,1],data_3_ok[:,2], color = "green", alpha = 0.25)
ax.scatter([0.0], [0.0], [0.0], color = "blue")
plt.title("Configuration = [1,-1,-1]")
plt.show()